In [ ]:
import joblib
import numpy as np
import pandas as pd

# Load saved components
loaded_model = joblib.load("fraud_detection_model_lg.pkl")
loaded_threshold = np.load("best_threshold_lg.npy")
loaded_scaler = joblib.load("scaler_lg.pkl")
cost_sensitive_model = joblib.load("cost_sensitive_xgb_model.pkl")
best_model = joblib.load("best_cost_sensitive_xgb.pkl")

In [ ]:
X_manual_sampled = pd.read_csv('manual_test_sample_rawd.csv')
y_manual_sampled = pd.read_csv('manual_test_sample_labelsd.csv')


X_manual_sampleds = pd.read_csv('manual_test_sample_rawds.csv')
y_manual_sampleds = pd.read_csv('manual_test_sample_labelsds.csv')

X_manual_sampledss = pd.read_csv('manual_test_sample_rawdss.csv')
y_manual_sampledss = pd.read_csv('manual_test_sample_labelsdss.csv')

X_manual_sampledssd = pd.read_csv('manual_test_sample_rawdssd.csv')
y_manual_sampledssd = pd.read_csv('manual_test_sample_labelsdssd.csv')

X_manual_samplessdd = pd.read_csv("manual_test_sample_rawdssdd.csv")
y_manual_samplessdd = pd.read_csv("manual_test_sample_labelsdssdd.csv")

In [ ]:
X_manual_sampledss

In [ ]:
y_manual_sampledss

In [ ]:
# Apply the same scaling to 'Amount' and 'Time' (if using new data)
# X_manual_sampled[['Amount', 'Time']] = loaded_scaler.transform(X_manual_sampled[['Amount', 'Time']])

# Apply the same scaling to 'Amount' and 'Time' (if using new data)
# X_manual_sampleds[['Amount', 'Time']] = loaded_scaler.transform(X_manual_sampleds[['Amount', 'Time']])


# Apply the same scaling to 'Amount' and 'Time' (if using new data)
# X_manual_sampledss[['Amount', 'Time']] = loaded_scaler.transform(X_manual_sampledss[['Amount', 'Time']])

# Apply the same scaling to 'Amount' and 'Time' (if using new data)
X_manual_sampledssd[['Amount', 'Time']] = loaded_scaler.transform(X_manual_sampledssd[['Amount', 'Time']])


In [ ]:
# Get fraud probabilities from the loaded model
# y_probs = loaded_model.predict_proba(X_manual_samplessdd)[:, 1]
# y_probss = cost_sensitive_model.predict_proba(X_manual_samplessdd)[:, 1]
y_probsss = best_model.predict_proba(X_manual_samplessdd)[:, 1]

# # Apply the best threshold for classification
# predicted_labels = (y_probs >= loaded_threshold).astype(int)

new_threshold = 0.8922086764891026 # 0.31641  # Example: Lowering threshold
predicted_labelss = (y_probsss >= new_threshold).astype(int)

# Create a DataFrame for manual validation using actual class labels from y_manual_sampled
manual_validation_df = y_manual_samplessdd  .copy()  # Copy the actual labels
manual_validation_df.rename(columns={'Class': 'Actual_Class'}, inplace=True)

# Add predictions and fraud probabilities
manual_validation_df['Predicted_Class'] = predicted_labelss  # Model predictions
manual_validation_df['Fraud_Probability'] = y_probsss  # Model confidence scores

# Select only relevant columns for display
manual_validation_df = manual_validation_df[['Actual_Class', 'Predicted_Class', 'Fraud_Probability']]


In [ ]:
import pandas as pd

# Ensure all rows & columns are visible
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Avoid wrapping

# Display first few rows for inspection
print("\n✅ Manual Validation Sample:")
print(manual_validation_df.head(100))  # Adjust number as needed


In [ ]:
# Evaluate the predictions
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, average_precision_score

precision = precision_score(y_manual_samplessdd, predicted_labelss)
recall = recall_score(y_manual_samplessdd, predicted_labelss)
f1 = f1_score(y_manual_samplessdd, predicted_labelss)
auc_pr = average_precision_score(y_manual_samplessdd, y_probsss)

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')
print(f'AUC-PR: {auc_pr:.4f}')

# Print classification report
print(classification_report(y_manual_samplessdd, predicted_labelss))

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_curve

# Get fraud probabilities from model
y_probs = loaded_model.predict_proba(X_manual_samplessdd)[:, 1]

# Compute precision-recall curve
precisions, recalls, thresholds = precision_recall_curve(y_manual_samplessdd, y_probs)

# Find the best threshold (balance between precision & recall)
f1_scores = 2 * (precisions * recalls) / (precisions + recalls)
best_threshold = thresholds[np.argmax(f1_scores)]

print(f"Optimal Threshold: {best_threshold:.5f}")


In [ ]:
from xgboost import XGBClassifier

# Adjust scale_pos_weight to focus on fraud detection
cost_sensitive_model = XGBClassifier(scale_pos_weight=50, random_state=42)  
cost_sensitive_model.fit(X_train, y_train)

# Evaluate the new model
y_probs_cs = cost_sensitive_model.predict_proba(X_manual_sampled)[:, 1]
y_pred_cs = (y_probs_cs >= best_threshold).astype(int)

print(classification_report(y_manual_sampled, y_pred_cs))

Another Level Check

In [ ]:
import pandas as pd

# Define review thresholds
review_lower = 0.3  # Lower bound for review
review_upper = 0.7  # Upper bound for review

# Apply thresholding logic
manual_validation_df['Action'] = "Approve"  # Default action

# If fraud probability is high, block transaction
manual_validation_df.loc[manual_validation_df['Fraud_Probability'] >= 0.9, 'Action'] = "Block"

# If fraud probability is in the review range, send for review
manual_validation_df.loc[
    (manual_validation_df['Fraud_Probability'] >= review_lower) & 
    (manual_validation_df['Fraud_Probability'] <= review_upper),
    'Action'
] = "Review"

# Display flagged transactions
print("\n✅ Transactions with Actions:")
print(manual_validation_df[['Actual_Class', 'Predicted_Class', 'Fraud_Probability', 'Action']])
